# Welcome to Catplat CLI tutorial

Catplat is a high throughput screening for heterogenous catalytic descriptors, with database storage functionality

## 1. Setting up and installing catplat

### Set up environment
Catplat is developed and build on top of pyatoms. As such, we would have to first set up pyatoms. ....

For convenience, we have set up the environment with the necessary dependencies and packages on ACRC and NSCC supercomputers. Simply activate the environment to start using catplat.

#note
still need to pip install . in catplat folder?

In [ ]:
!conda activate ______

## 2. Catplat CLI commands

There are three main commands for using catplat. They are catplat config, catplat calculate and catplat retrieve.
catplat config: 
catplat calculate:
catplat retrieve:

For more details for each command, simply run catplat __script_name__ --help.

### 2.1  Catplat Config

catplat config is used to configure catplat. Using this command, we can set up settings such as the project name, calculation path, database path.

Options:

In [2]:
!catplat config --project wgs --calc-path ~/catplat/wgs/calc --db-path ~/catplat/wgs/db

/bin/bash: catplat: command not found


### 2.2 Catplat Calculate

### 2.3 Catplat Retrieve